In [3]:
!pip install nltk scikit-learn contractions

In [4]:
# performing NER using spaCy( NLP library)

!pip install spaCy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!pip install clean-text

In [6]:
!pip install emoji

In [7]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import re
import nltk
import spacy
import contractions
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import ngrams
from cleantext import clean
import emoji

In [8]:
#load dataset
df = pd.read_csv("/content/db1.csv", quoting=csv.QUOTE_MINIMAL)
df

,ID,Resume_str,Category,job_id,title,description,ATS_score
0,18176523,SENIOR INFORMATION TECHNOLOGY MANAGER...,INFORMATION-TECHNOLOGY,3905367422,Trademark Attorney,Junior Trademark Associate\nOur client is a to...,21
1,18176523,SENIOR INFORMATION TECHNOLOGY MANAGER...,INFORMATION-TECHNOLOGY,3887888322,Delivery Driver / CDL A required / Seasonal,PBNA $25.75 / hour\n\nCLICK HERE to view our D...,24
2,18176523,SENIOR INFORMATION TECHNOLOGY MANAGER...,INFORMATION-TECHNOLOGY,3905243094,Senior Recruiter,Hit a glass ceiling in your earning potential ...,47
3,18176523,SENIOR INFORMATION TECHNOLOGY MANAGER...,INFORMATION-TECHNOLOGY,3905323971,Order Fulfillment Coordinator,Are you ready to be a crucial part of our dyna...,51
4,18176523,SENIOR INFORMATION TECHNOLOGY MANAGER...,INFORMATION-TECHNOLOGY,3903830212,Medical Assistant Urgent Care Per Diem,"As a physician-founded and led organization, e...",38
...,...,...,...,...,...,...,...
9995,57088974,FINANCE MANAGER Summary ...,FINANCE,3904500805,Markit/Cadis MDM Developer,Job Summary for Markit/Cadis MDM Developer:Our...,34
9996,57088974,FINANCE MANAGER Summary ...,FINANCE,3905330406,Retail Parts Pro,Job Description\n\nWhat is a Retail Parts Pro?...,37
9997,57088974,FINANCE MANAGER Summary ...,FINANCE,3904929128,Site Reliability Engineer - Federal - 3rd Shift,"Company Description\n\nAt ServiceNow, our tech...",19
9998,57088974,FINANCE MANAGER Summary ...,FINANCE,3905631040,Travel RN - Case Management,Procare USA is seeking a travel nurse RN Case ...,14


In [9]:
#Load NLP models
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
#Define patterns
url_pattern = re.compile(r'http[s]?://(?:[?[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
ip_pattern = re.compile(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\b')
mention_pattern = re.compile(r'@\w+')
stop_words_nltk = set(stopwords.words('english'))
stemmer =PorterStemmer()

In [11]:
#Text cleaning functions
def format_and_clean(text):
  text = re.sub(url_pattern,'', text) # Format URLs
  text = re.sub(email_pattern, '', text) #validate emails
  text = re.sub(ip_pattern, '', text) #validate IPs
  text = re.sub(mention_pattern, '', text) #Format mentions
  return text
  #Apply preprocessing to each resume and job description using NLTK stop words removal
df['Resume_str'] = df['Resume_str'].apply(format_and_clean)
df['description'] = df['description'].apply(format_and_clean)

In [12]:
#Function to remove emojis
def remove_emojis(text):
  return emoji.replace_emoji(text, replace="")
df['Resume_str'] = df['Resume_str'].apply(remove_emojis)
df['description'] = df['description'].apply(remove_emojis)

In [13]:
def expand_contractions(text):
  return contractions.fix(text)
df['Resume_str'] = df['Resume_str'].apply(expand_contractions)
df['description'] = df['description'].apply(expand_contractions)

In [14]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
def remove_symbols(text):
  pattern = r'[^\w\s]'
  return re.sub(pattern, '', text)

df['Resume_str'] = df['Resume_str'].apply(remove_symbols)
df['description'] = df['description'].apply(remove_symbols)

In [17]:
#Function to remove stop words using NLTK
stop_words_nltk = set(stopwords.words('english'))

def remove_stopwords_nltk(text):
  tokens = nltk.word_tokenize(text)
  tokens = [token for token in tokens if token.lower() not in stop_words_nltk]
  return ' '.join(tokens)
df['Resume_str'] = df['Resume_str'].apply(remove_stopwords_nltk)
df['description'] = df['description'].apply(remove_stopwords_nltk)

In [18]:
#Function to remove stop words using spacy
def remove_stopwords_spacy(text):
  doc = nlp(text)
  tokens = [token.text for token in doc if not token.is_stop]
  return ' '.join(tokens)


In [19]:
def stem_text(text):
  tokens = nltk.word_tokenize(text)
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  return ''.join(stemmed_tokens)
df['Resume_str'] = df['Resume_str'].apply(stem_text)
df['description'] = df['description'].apply(stem_text)

In [20]:
# Function to apply lemmatization using spacy
def lemmatize_text(text):
  doc = nlp(text)
  lemmatized_tokens = [token.lemma_ for token in doc]
  return ''.join(lemmatized_tokens)
df['Resume_str'] = df['Resume_str'].apply(lemmatize_text)
df['description'] = df['description'].apply(lemmatize_text)

In [21]:
df

,ID,Resume_str,Category,job_id,title,description,ATS_score
0,18176523,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,INFORMATION-TECHNOLOGY,3905367422,Trademark Attorney,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,21
1,18176523,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,INFORMATION-TECHNOLOGY,3887888322,Delivery Driver / CDL A required / Seasonal,xxxxddddxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,24
2,18176523,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,INFORMATION-TECHNOLOGY,3905243094,Senior Recruiter,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,47
3,18176523,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,INFORMATION-TECHNOLOGY,3905323971,Order Fulfillment Coordinator,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,51
4,18176523,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,INFORMATION-TECHNOLOGY,3903830212,Medical Assistant Urgent Care Per Diem,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,38
...,...,...,...,...,...,...,...
9995,57088974,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,FINANCE,3904500805,Markit/Cadis MDM Developer,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,34
9996,57088974,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,FINANCE,3905330406,Retail Parts Pro,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,37
9997,57088974,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,FINANCE,3904929128,Site Reliability Engineer - Federal - 3rd Shift,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,19
9998,57088974,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,FINANCE,3905631040,Travel RN - Case Management,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx...,14


In [22]:
df['Resume_str'] = df['Resume_str'].str.lower()
df['description'] = df['description'].str.lower()

In [23]:
#Function to generate n-grams
def generate_ngrams(text, n):
  tokens = nltk.word_tokenize(text)
  return list(ngrams(tokens, n))

In [24]:
#Generate bigrams for resumes and job descriptions
df['Resume_bigrams'] = df['Resume_str'].apply(lambda x: generate_ngrams(x, 2))
df['description_bigrams'] = df['description'].apply(lambda x: generate_ngrams(x, 2))

In [25]:
#Using CountVectorizer for n-grams
vectorizer = CountVectorizer(ngram_range=(1,2)) # Unigrams and Bigrams
Resume_ngrams = vectorizer.fit_transform(df['Resume_str'])
description_ngrams = vectorizer.fit_transform(df['description'])


In [26]:
# Get feature names
features = vectorizer.get_feature_names_out()
print("\nFeatures (Unigrams and Bigrams):")
print(features)


Features (Unigrams and Bigrams):
['...']


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
resume_tfidf = tfidf_vectorizer.fit_transform(df['Resume_str'])
description_tfidf = tfidf_vectorizer.fit_transform(df['description'])

In [42]:
import torch
from transformers import BertTokenizer, BertModel

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode texts in batches
def encode_texts_in_batches(texts, batch_size=32):
    encoded_batches = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
        encoded_batches.append(encoded_input)
    return encoded_batches

# Function to get embeddings in batches
def get_embeddings_in_batches(encoded_batches):
    embeddings = []
    with torch.no_grad():
        for encoded_input in encoded_batches:
            outputs = model(**encoded_input)
            embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())
    return np.concatenate(embeddings, axis=0)

# Encode texts in batches
resume_batches = encode_texts_in_batches(df['Resume_str'].tolist())
description_batches = encode_texts_in_batches(df['description'].tolist())

# Get embeddings in batches
resume_embeddings = get_embeddings_in_batches(resume_batches)
description_embeddings = get_embeddings_in_batches(description_batches)

print("Resume Embeddings Shape:", resume_embeddings.shape)
print("Description Embeddings Shape:", description_embeddings.shape)

Resume Embeddings Shape: (10000, 768)
Description Embeddings Shape: (10000, 768)


In [44]:
import numpy as np

# Combine embeddings
X = np.hstack([resume_embeddings, description_embeddings])
y = df['ATS_score']  # Assuming you have a label column for supervised learning


from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [48]:
from sklearn.metrics import mean_squared_error
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE: {rmse}")

RMSE: 10.924811211183469
